In [2]:
#spark.stop()

In [10]:
import os, sys
import datetime as dt
DEVEL_SRC = os.path.join(os.environ.get('BDA_USER_HOME', ''), "src", "devel")
if DEVEL_SRC not in sys.path:
    sys.path.append(DEVEL_SRC)

USECASES_SRC = os.path.join(DEVEL_SRC, "use-cases") # TODO when - is removed, remove also this line and adapt imports
if USECASES_SRC not in sys.path: 
    sys.path.append(USECASES_SRC)
    
AMDOCS_SRC = os.path.join(DEVEL_SRC, "amdocs_informational_dataset") # TODO when - is removed, remove also this line and adapt imports
if AMDOCS_SRC not in sys.path: 
    sys.path.append(AMDOCS_SRC)
    
import pykhaos.utils.custom_logger as clogger
logging_file = os.path.join(os.environ.get('BDA_USER_HOME', ''), "logging",
                                    "out_" + dt.datetime.now().strftime("%Y%m%d_%H%M%S") + ".log")
logger = clogger.configure_logger(log_filename=logging_file, std_channel=sys.stderr, logger_name="")
logger.info("Logging to file {}".format(logging_file))    
    
    
from project.project_generic import Project


import pykhaos.utils.notebooks as nb

project_obj = Project("encuestas bajas", "encuestas bajas")   

RUNNING_FROM_NOTEBOOK = nb.isnotebook()
import matplotlib.pyplot as plt
if RUNNING_FROM_NOTEBOOK:
    %load_ext autoreload
    %autoreload 2
    %matplotlib inline
    EXTERNAL_LIB = os.path.join(os.environ.get('BDA_USER_HOME', ''), "lib", "external_libs")
    sys.path.append(EXTERNAL_LIB)
    %load_ext jupyternotify
    
#logger = my_project.logger

if not RUNNING_FROM_NOTEBOOK:
    args = my_project.arg_parser()

20181213-195030 [INFO ] Logging to file /var/SP/data/home/csanc109/logging/out_20181213_195030.log


_initialize spark
Ended spark session: 0.0331211090088 secs | default parallelism=4
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<IPython.core.display.Javascript object>

In [11]:
spark=project_obj.spark

In [12]:
%load_ext autoreload
%autoreload 2
%matplotlib inline 
from churn.datapreparation.config import Config



default_filename = os.path.join(USECASES_SRC, "churn", "datapreparation", "input", "dp_ccc_model.yaml")


from churn.config_manager.ccc_model_config_mgr import  CCCmodelConfig
config_obj = CCCmodelConfig(default_filename)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Reading config from file /var/SP/data/home/csanc109/src/devel/use-cases/churn/datapreparation/input/dp_ccc_model.yaml
Reading internal config from file /var/SP/data/home/csanc109/src/devel/use-cases/churn/config_manager/config/internal_config_ccc_model.yaml
----- CHECKING INPUT PARAMETERS CCCmodel------
{'agg_by': 'msisdn',
 'ccc_days': -60,
 'closing_day': 20180430,
 'end_port': 20180531,
 'internal_config_file': '/var/SP/data/home/csanc109/src/devel/use-cases/churn/config_manager/config/internal_config_ccc_model.yaml',
 'labeled': True,
 'level': 'nif',
 'model_target': 'comercial',
 'save_car': True,
 'sources': {'ids': {'address': False,
                     'billing': True,
                     'call_centre_calls': True,
                     'campaigns': False,
                     'customer': True,
                     'customer_aggregations': True,
                     'customer_penalties': F

In [31]:
from churn.datapreparation.engine.bajas_data_loader import load_reasons_encuestas
from pyspark.sql.functions import concat, lpad, col
df_encuestas = load_reasons_encuestas(spark, config_obj=None).withColumnRenamed("NIF_reasons", "NIF")
df_encuestas = df_encuestas.withColumnRenamed("reason", "label")
df_encuestas = df_encuestas.withColumn("portout_encuesta", concat(col('year'), lpad(col('month'), 2, '0'), lpad(col('day'), 2, '0')))
df_encuestas = df_encuestas.select("NIF", "yearmonth", "REASON_ENCUESTA", "portout_encuesta")


Returning all information churn_reason...


In [5]:

CAT_PRECIO = "PRECIO"
CAT_TERMINAL = "TERMINAL"
CAT_CONTENIDOS = "CONTENIDOS"
CAT_COMERCIAL = "COMERCIAL"
CAT_NO_COMERCIAL = "NO_COMERCIAL"
CAT_SERV_ATENCION = "SERVICIO/ATENCION"
CAT_TECNICO = "TECNICO"
CAT_BILLING = "BILLING"
CAT_FRAUD = "FRAUD"
NO_PROB = "NO_PROB"
NA = "NA"

# ADD CATEGORIES TO CALLS
REASON1_PRECIO = ["Cliente quiere pagar menos", "Clte quiere pagar menos"]


def __set_categories(df_all):

    start_time = time.time()

    df_all = df_all.withColumn("is_fraud", when(col("INT_RAZON").rlike("(?i)enga.o comercial|venta enga.osa") ,1).otherwise(0))
    df_all = df_all.withColumn("Queja_Trato", when(col("INT_SUBTIPO").isin(["Queja trato", "Quejas sevicios de atencion"])
                                    ,1).otherwise(0))

    df_all = df_all.withColumn("CATEGORY_2",
                               # - - - - - - - - - PRECIO
                               when(col('INT_TIPO').isin(REASON1_PRECIO) , CAT_PRECIO)
                               # - - - - - - - - - SERVICIO/ATENCION
                               .when(lower(col("INT_SUBTIPO")).isin
                                   (["Queja trato".lower(), "Quejas sevicios de atencion".lower()]) ,CAT_SERV_ATENCION)
                               # - - - - - - - - - ENGAÑO
                               .when(col("INT_RAZON").rlike("(?i)enga.o comercial|venta enga.osa"), CAT_FRAUD)
                               # - - - - - - - - - TECNICO
                               .when(col('INT_TIPO').rlike('(?i)^Averia') ,CAT_TECNICO)
                               .when((col('INT_TIPO').rlike('(?i)^Transferencia') )&
                                   (col("INT_SUBTIPO").rlike('(?i)^Aver.as')), CAT_TECNICO)
                               .when(col('INT_TIPO').rlike(
                                   '(?i)^Inc Provis.*Neba|^Inc Provision Fibra|^Inc Provision DSL|^Incidencia%ecnica|^Incidencia%SGI|^Inc|^Incidencia'),
                                     CAT_TECNICO)
                               .when(col('INT_TIPO').rlike('(?i)^Consulta tec'), CAT_TECNICO)
                               .when(col("Raw_Resultado").rlike(
                                   "Raw_Resultado_Escalo|Raw_Resultado_Envio_tecnico|Raw_Resultado_Transferencia|Raw_Resultado_Reclamacion"),
                                     CAT_TECNICO)
                               # - - - - - - - - - FACTURA
                               .when(col('INT_TIPO').rlike('(?i)factura'), CAT_BILLING)
                               # - - - - - - - - - TERMINAL
                               .when(col("INT_TIPO") == "TERMINAL", CAT_TERMINAL)
                               # - - - - - - - - - NO_PROB :)
                               .when((col("INT_TIPO") == "INFORMACION") | ((col("INT_TIPO") == "TRANSFERENCIA") & (
                                           col("Bucket") == "Other customer information management")),
                                     NO_PROB).otherwise(NA)
                               )  # end

    df_all = df_all.na.fill(NA, subset=["CATEGORY_2"])

    df_all = (df_all.withColumn("CATEGORY_1",
                                when(col('CATEGORY_2').isin([CAT_PRECIO, CAT_TERMINAL, CAT_CONTENIDOS]), CAT_COMERCIAL)
                                .when(col('CATEGORY_2').isin([CAT_SERV_ATENCION, CAT_TECNICO, CAT_BILLING, CAT_FRAUD]),
                                      CAT_NO_COMERCIAL)
                                .otherwise(col('CATEGORY_2'))))

    print("Elapsed time _set_categories = {} minutes".format((time.time()-start_time)/60.0))
    # FALTA
    # col('Raw_Productos') hay que desglosarlo
    return df_all




def __aggregate_ccc_calls_by(df_all, process_date, agg_by="nif"):

    start_time = time.time()


    print("Calling to __aggregate_ccc_calls_by_{}.... {}".format(agg_by, process_date))

    df_agg = (df_all
              .withColumn("fx_interaction",
                          concat(col('year'), lpad(col('month'), 2, '0'), lpad(col('day'), 2, '0')))
              .withColumn("tuples", struct(["fx_interaction", "Bucket"]))
              .groupby(agg_by)
              .agg(sql_count(lit(1)).alias("num_interactions"),
                   sql_min(col('fx_interaction')).alias("first_interaction"),
                   sql_max(col('fx_interaction')).alias("latest_interaction"),
                   sql_max(col("tuples")).alias("tuples_max"),  # latest interaction: [max(fx_interaction), bucket]
                   sql_min(col("tuples")).alias("tuples_min"),  # first interaction: [min(fx_interaction), bucket]
                   collect_list('Bucket').alias("bucket_list"),
                   collect_set('Bucket').alias("bucket_set"),
                   sql_sum("Bucket_NA").alias("num_NA_buckets"),
                   sql_sum("IVR").alias("num_ivr_interactions"),
                   collect_list('CATEGORY_1').alias("cat1_list"),
                   collect_list('CATEGORY_2').alias("cat2_list"),
            ))

    df_agg = df_agg.withColumn("ref_date", format_date(lit(process_date)))
    df_agg = (df_agg.withColumn("bucket_1st_interaction", col("tuples_min")["Bucket"])
                    .withColumn("bucket_latest_interaction", col("tuples_max")["Bucket"])
                    .withColumn("nb_diff_buckets", size("bucket_set"))
                   .drop(*['tuples_max', 'tuples_min'])
              )

    for cc in ["first_interaction", "latest_interaction"]:
        df_agg = (df_agg.withColumn("fx_{}".format(cc), format_date(cc, filter_dates_1900=True))  # days before 1900 converted to None
                        .withColumn("days_since_{}".format(cc), compute_diff_days("fx_{}".format(cc), "ref_date")))

    df_agg = df_agg.drop("ref_date")

    def get_mode_problems(lst):
        # filter NA y NO_PROB
        if not lst: return None
        lst=[ll for ll in lst if ll and ll!=NO_PROB and ll!="NA"]
        if not lst: return None
        dd = Counter(lst).most_common(2)
        return dd[0][0]
    get_mode_udf = udf(lambda lst: get_mode_problems(lst), StringType())

    df_agg = (df_agg.withColumn("CAT1_MODE", when(coalesce(size(col("cat1_list")), lit(0)) == 0, "None").otherwise(get_mode_udf(col("cat1_list"))))
                    .withColumn("CAT2_MODE", when(coalesce(size(col("cat2_list")), lit(0)) == 0, "None").otherwise(get_mode_udf(col("cat2_list")))))



    df_cat_1 = df_all.groupby(agg_by).pivot("CATEGORY_1", values=[CAT_COMERCIAL, CAT_NO_COMERCIAL]).agg(sql_count("*")).fillna(0)
    df_cat_2 = df_all.groupby(agg_by).pivot("CATEGORY_2", values=[CAT_PRECIO, CAT_TERMINAL, CAT_CONTENIDOS, CAT_SERV_ATENCION,
CAT_TECNICO, CAT_BILLING, CAT_FRAUD, NO_PROB]).agg(sql_count("*")).fillna(0)

    df_agg = df_agg.join(df_cat_1, [agg_by], how="left").join(df_cat_2,  [agg_by], how="left")

    df_agg = (df_agg.withColumn("TOTAL_COMERCIAL", sum_horizontal([CAT_PRECIO, CAT_TERMINAL, CAT_CONTENIDOS]))
                    .withColumn("TOTAL_NO_COMERCIAL", sum_horizontal([CAT_SERV_ATENCION, CAT_TECNICO, CAT_BILLING, CAT_FRAUD])))


    print("Elapsed time __aggregate_ccc_calls_by_{} = {} minutes".format(agg_by, (time.time()-start_time)/60.0))

    return df_agg


In [6]:
from churn.datapreparation.general.data_loader import get_port_requests_table


from pykhaos.utils.date_functions import move_date_n_days

from pyspark.sql.functions import col

from amdocs_informational_dataset.engine.call_centre_calls import CallCentreCalls

from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import col, concat, concat_ws, date_format, dayofmonth, format_string, from_unixtime, length, \
	lit, lower, lpad, month, regexp_replace, translate, udf, unix_timestamp, year, when, upper, collect_set, collect_list, \
    count as sql_count, min as sql_min, max as sql_max, struct, size, coalesce, sum as sql_sum
from pyspark.sql.types import DateType, IntegerType, StringType, StructField, StructType
from pyspark.sql.utils import AnalysisException
from pykhaos.utils.pyspark_utils import format_date, compute_diff_days
from collections import Counter
import time
from engine.general_functions import sum_horizontal


In [21]:
# def amdocs_table_reader(spark, table_name, closing_day, new=True):
#     print("amdocs_table_reader", table_name, closing_day)
#     path_car = "amdocs_inf_dataset" if new else "amdocs_ids"
#     table_name = '/data/udf/vf_es/{}/{}/year={}/month={}/day={}'.format(path_car, table_name, int(closing_day[:4]),
#                                                                                                     int(closing_day[4:6]),
#                                                                                                     int(closing_day[6:]))
#     print("Loading {}".format(table_name))
#     df_src = spark.read.load(table_name)
#     return df_src

# from pykhaos.utils.pyspark_utils import union_all

# df_s = []

# for closing_day in ["20180331", "20180430"]:


#     df_customer = (amdocs_table_reader(spark, "customer", closing_day)
#                  #.where(col("clase_cli_cod_clase_cliente") == "RS")  # customer
#                  #.where(col("cod_estado_general").isin(["01", "09"]))  # customer
#                  .select("num_cliente", "cod_estado_general", "clase_cli_cod_clase_cliente", "nif_cliente")
#                  .withColumnRenamed("num_cliente", "num_cliente_customer"))

#     df_service = (amdocs_table_reader(spark, "service", closing_day)
#                   #.where(~col("srv_basic").isin(["MRSUI", "MPSUI"]))  # service
#                   #.where(col("rgu").isNotNull())
#                   .select("msisdn", "num_cliente", "campo2", "rgu", "srv_basic")
#                   .withColumnRenamed("num_cliente", "num_cliente_service"))

#     df_services = df_customer.join(df_service,
#                   on=(df_customer["num_cliente_customer"] == df_service["num_cliente_service"]), how="inner") # intersection

#     df_services = df_services.withColumnRenamed("nif_cliente", "nif")

#     df_services = df_services.select("msisdn", "nif")

#     df_s.append(df_services)
    
# df_services = union_all(df_s)

# df_services = df_services.dropDuplicates(["msisdn"])

# df_services_encuestas = df_services.join(df_encuestas, ["NIF"], how="right")

# #df_services_encuestas = df_services_encuestas.where(col("msisdn").isNotNull())

('amdocs_table_reader', 'customer', '20180331')
Loading /data/udf/vf_es/amdocs_inf_dataset/customer/year=2018/month=3/day=31
('amdocs_table_reader', 'service', '20180331')
Loading /data/udf/vf_es/amdocs_inf_dataset/service/year=2018/month=3/day=31
('amdocs_table_reader', 'customer', '20180430')
Loading /data/udf/vf_es/amdocs_inf_dataset/customer/year=2018/month=4/day=30
('amdocs_table_reader', 'service', '20180430')
Loading /data/udf/vf_es/amdocs_inf_dataset/service/year=2018/month=4/day=30


In [22]:
df_services_encuestas.dropDuplicates(["NIf"]).count()

6036

In [32]:
min_portout = df_encuestas.select(sql_min("portout_encuesta").alias("min_portout_date")).rdd.map(
    lambda x: x["min_portout_date"]).collect()[0]
max_portout = df_encuestas.select(sql_max("portout_encuesta").alias("max_portout_date")).rdd.map(
    lambda x: x["max_portout_date"]).collect()[0]

In [38]:
max_portout

u'20180901'

In [40]:
n_ccc=config_obj.get_ccc_range_duration()

min_portout = df_encuestas.select(sql_min("portout_encuesta").alias("min_portout_date")).rdd.map(
    lambda x: x["min_portout_date"]).collect()[0]
max_portout = df_encuestas.select(sql_max("portout_encuesta").alias("max_portout_date")).rdd.map(
    lambda x: x["max_portout_date"]).collect()[0]

yyyymmdd_min_portout_date = str(min_portout.split(" ")[0].replace("-", ""))
yyyymmdd_max_portout_date = str(max_portout.split(" ")[0].replace("-", ""))

ccc_start_date = move_date_n_days(yyyymmdd_min_portout_date, n=n_ccc-30, str_fmt="%Y%m%d")
ccc_end_date  = move_date_n_days(yyyymmdd_max_portout_date, n=30, str_fmt="%Y%m%d")


In [48]:
%%notify -m "encuestas: ended dp "


print(ccc_start_date, ccc_end_date)
ccc = CallCentreCalls(spark)
ccc.prepareFeatures(ccc_end_date, ccc_start_date)
df_all = ccc.all_interactions
df_all = df_all.withColumn("fx_interaction", concat(col('year'), lpad(col('month'), 2, '0'), lpad(col('day'), 2, '0')))
#df_all = df_all.drop("year", "month", "day")

# CALLS OF POLLS USERS
df_all_encuestas = df_all.join(df_encuestas, ["NIF"], how="right") 

print("df_all_encuestas unique NIFS={}".format(df_all_encuestas.dropDuplicates(["NIF"]).count()))
# ('20180101', '20181001')
# df_all_encuestas unique NIFS=6036


('20180101', '20181001')
df_all_encuestas unique NIFS=6036


<IPython.core.display.Javascript object>

In [49]:
df_all_encuestas.dtypes

[('NIF', 'string'),
 ('msisdn', 'string'),
 ('partitioned_month', 'string'),
 ('year', 'int'),
 ('month', 'int'),
 ('day', 'int'),
 ('INT_Tipo', 'string'),
 ('INT_Subtipo', 'string'),
 ('INT_Razon', 'string'),
 ('INT_Resultado', 'string'),
 ('DIRECTION', 'string'),
 ('x_workgroup', 'string'),
 ('source', 'string'),
 ('TYPE_TD', 'string'),
 ('Sub_Bucket', 'string'),
 ('Raw_Pagar_menos', 'string'),
 ('Raw_Cobro', 'string'),
 ('Raw_Precios', 'string'),
 ('Raw_Averia', 'string'),
 ('Raw_Alta', 'string'),
 ('Raw_Desactivacion', 'string'),
 ('Raw_Factura', 'string'),
 ('Raw_Ofrecimiento', 'string'),
 ('Bucket_Sub_Bucket', 'string'),
 ('Raw_Incidencia', 'string'),
 ('Raw_Consulta', 'string'),
 ('Raw_Informacion', 'string'),
 ('Raw_Portabilidad', 'string'),
 ('Raw_Cierre', 'string'),
 ('Raw_Productos', 'string'),
 ('Raw_Resultado', 'string'),
 ('Raw_Provision', 'string'),
 ('Bucket', 'string'),
 ('Raw_Transferencia', 'string'),
 ('Raw_Baja', 'string'),
 ('IVR', 'int'),
 ('Bucket_NA', 'int'),
 

In [53]:
df_all_encuestas = (df_all_encuestas.withColumn("fx_fx_interaction", format_date("fx_interaction", filter_dates_1900=True))  # days before 1900 converted to None
                    .withColumn("fx_portout_encuestas", format_date("portout_encuesta", filter_dates_1900=True))
               .withColumn("days_since_fx_interaction", compute_diff_days("fx_fx_interaction", "fx_portout_encuestas")))

df_all_encuestas.select("days_since_fx_interaction", "fx_fx_interaction", "portout_encuesta").show(10,False)

+-------------------------+-------------------+----------------+
|days_since_fx_interaction|fx_fx_interaction  |portout_encuesta|
+-------------------------+-------------------+----------------+
|3.0                      |2018-06-28 00:00:00|20180701        |
|237.0                    |2018-01-07 00:00:00|20180901        |
|87.0                     |2018-03-06 00:00:00|20180601        |
|154.0                    |2018-02-28 00:00:00|20180801        |
|154.0                    |2018-02-28 00:00:00|20180801        |
|10.0                     |2018-03-22 00:00:00|20180401        |
|10.0                     |2018-03-22 00:00:00|20180401        |
|180.0                    |2018-01-02 00:00:00|20180701        |
|137.0                    |2018-01-15 00:00:00|20180601        |
|137.0                    |2018-01-15 00:00:00|20180601        |
+-------------------------+-------------------+----------------+
only showing top 10 rows



In [50]:

df_all_encuestas = (df_all_encuestas.withColumn("fx_fx_interaction", format_date("fx_interaction", filter_dates_1900=True))  # days before 1900 converted to None
                    .withColumn("fx_portout_encuestas", format_date("portout_encuesta", filter_dates_1900=True))
               .withColumn("days_since_fx_interaction", compute_diff_days("fx_fx_interaction", "fx_portout_encuestas")))

df_all_encuestas = df_all_encuestas.where( (col("days_since_fx_interaction")>=0) & (col("days_since_fx_interaction")<=abs(n_ccc)))

df_all_encuestas = __set_categories(df_all_encuestas)

df_agg_all_ccc = __aggregate_ccc_calls_by(df_all_encuestas, ccc_end_date, agg_by="NIF")

print("df_agg_all_ccc={}".format(df_agg_all_ccc.count()))

df_agg_all_ccc = df_agg_all_ccc.drop('bucket_list', 'bucket_set', 'cat1_list', 'cat2_list')

df_agg_all_ccc = df_encuestas.join(df_agg_all_ccc, ["NIF"], how="left")


Elapsed time _set_categories = 0.00739806890488 minutes
Calling to __aggregate_ccc_calls_by_NIF.... 20181001
Elapsed time __aggregate_ccc_calls_by_NIF = 0.040777250131 minutes
df_agg_all_ccc=1502


In [43]:
df_agg_all_ccc.count()

6036

In [52]:
df_agg_all_ccc.where(col("num_interactions")>0).count()

1502

#### WRITE

In [34]:
from churn.datapreparation.engine.bajas_data_loader import save_results
df_agg_all_ccc = df_agg_all_ccc.repartition(1)
save_results(df_agg_all_ccc,config_obj, csv=True)

df_20180501_20180531_c20180430_n60_comercial_msisdn
True
comercial
Built filename: /data/udf/vf_es/churn/encuestas/comercial/df_20180501_20180531_c20180430_n60_comercial_msisdn
Directory '/data/udf/vf_es/churn/encuestas/comercial' does not exist. Creating...
create_directory return_state=''
Created directory returned True
Started to save...
Saved df successfully - '/data/udf/vf_es/churn/encuestas/comercial/df_20180501_20180531_c20180430_n60_comercial_msisdn'
Saved df in '/data/udf/vf_es/churn/encuestas/comercial/df_20180501_20180531_c20180430_n60_comercial_msisdn' format csv
Directory '/data/udf/vf_es/churn/encuestas/comercial/df_20180501_20180531_c20180430_n60_comercial_msisdn/yaml' does not exist. Creating...


20181213-004731 [INFO ] Moving '/var/SP/data/home/csanc109/src/devel/use-cases/churn/datapreparation/input/dp_ccc_model.yaml' to hdfs '/data/udf/vf_es/churn/encuestas/comercial/df_20180501_20180531_c20180430_n60_comercial_msisdn/yaml'


create_directory return_state=''
Running system command: hdfs dfs -put /var/SP/data/home/csanc109/src/devel/use-cases/churn/datapreparation/input/dp_ccc_model.yaml /data/udf/vf_es/churn/encuestas/comercial/df_20180501_20180531_c20180430_n60_comercial_msisdn/yaml


20181213-004736 [INFO ] Moving '/var/SP/data/home/csanc109/src/devel/use-cases/churn/config_manager/config/internal_config_ccc_model.yaml' to hdfs '/data/udf/vf_es/churn/encuestas/comercial/df_20180501_20180531_c20180430_n60_comercial_msisdn/yaml'


Running system command: hdfs dfs -put /var/SP/data/home/csanc109/src/devel/use-cases/churn/config_manager/config/internal_config_ccc_model.yaml /data/udf/vf_es/churn/encuestas/comercial/df_20180501_20180531_c20180430_n60_comercial_msisdn/yaml


'/data/udf/vf_es/churn/encuestas/comercial/df_20180501_20180531_c20180430_n60_comercial_msisdn'